In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def convert_to_ratio(df):
    return (df.T / df.sum(axis=1)).T

In [3]:
mse = pd.read_csv('./gradients/mse/gradients.csv', header=None)
cse = pd.read_csv('./gradients/cse/gradients.csv', header=None)
mse_weight = pd.read_csv('./gradients/mse/weight_gradients.csv', header=None) / 32
cse_weight = pd.read_csv('./gradients/cse/weight_gradients.csv', header=None) / 32

mse = convert_to_ratio(mse)
cse = convert_to_ratio(cse)
mse_weight = convert_to_ratio(mse_weight)
cse_weight = convert_to_ratio(cse_weight)

In [4]:
x = np.arange(len(mse))
colors = ['#264653', '#2A9D8F', '#E9C46A', '#F4A261', '#E76F51']
colors2 = ['#f15854', '#faa43a', '#60bd68', '#5da5da', '#b276b2']

In [7]:
def stackplot(dfs, aggregate, step, figsize, colors):
    rc = {"font.family" : "serif", "mathtext.fontset" : "stix"}
    plt.rcParams.update(rc)
    plt.rcParams["font.serif"] = ["Times New Roman"] + plt.rcParams["font.serif"]
    plt.rc('text', usetex=True)
    
    fig = plt.figure(figsize=figsize, dpi=1200)
    
    n_dfs = len(dfs)
    n_rows = len(dfs[0])
    n_samples = n_rows // step
    sub_titles = ['LeNet5', 'LeNet5+']
    labels = ['conv1', 'conv2', 'fc1', 'fc2', 'fc3']
    ylabels = '$\frac{\partial J / \partial w^l}{\sum_i \partial J}$'
    xlabels = 'epoch'
    
    for i in range(n_dfs):
        ax = fig.add_subplot(1, n_dfs, i + 1)
        samples = []
        for j in range(n_samples):
            beg = step * j
            end = step * (j + 1)
            if aggregate is True:
                samples.append(dfs[i].iloc[beg:end, :].mean())
            else:
                samples.append(dfs[i].iloc[beg, :])
        
        x = np.arange(0, n_samples)
        y = np.array(samples).T
        
        ax.stackplot(x, y, labels=labels, colors=colors, alpha=0.9)
        ax.set_ylim(0., 1.0)
        ax.set_xlim(0., len(x))
        
        xticks = np.linspace(0, n_samples, 8)
        xticklabels = [f'{int(t * step // 1000)}k' for t in xticks]
        
        ax.set_xticks(xticks)
        ax.set_xticklabels(xticklabels, fontsize=12)
        ax.tick_params(axis='y', labelsize=12)
        
        ax.set_title(sub_titles[i], fontsize=18, pad=12)
        ax.set_ylabel(r'$ ||\delta^l||_2 \bigg/ \sum_{i=1}^{5} \ ||\delta^i||_2 $', fontsize=14, labelpad=12)
        ax.set_xlabel('Minibatch number', fontsize=16, labelpad=10)
        #ax.grid(axis='y', color='k', alpha=0.6)
        
        handle, label = ax.get_legend_handles_labels()
        ax.legend(reversed(handle), reversed(label), fontsize=12)
    
    plt.subplots_adjust(wspace=0.25)
    fig.savefig("gradient_proportion.pdf", bbox_inches='tight')
    
    plt.show()

In [8]:
stackplot([mse, cse], True, 150, (16, 4), colors)